In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
def show(df, limit = 5):
    return df.toPandas().head(limit)

# Events

In [8]:
events = spark.read.csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_glory.csv", header=True)

In [10]:
events = events.withColumn("date", F.to_date(F.col("date"), "MMMM d, yyyy"))\
    .withColumn("attendance", F.regexp_replace(F.col("attendance"), ",", ""))\
    .withColumn("attendance", F.col("attendance").cast(T.IntegerType()))\
    .withColumn("location", F.when(F.col("location") == "—", F.lit(None)).otherwise(F.col("location")))

In [13]:
events = events.withColumn("location", F.split(F.col("location"), ","))\
    .withColumn("city", F.when(F.size(F.col("location")) == 3, F.element_at(F.col("location"), 1)))\
    .withColumn("state", F.when(
        F.size(F.col("location")) == 3, F.element_at(F.col("location"), 2)
        ).otherwise(F.element_at(F.col("location"), 1))
        )\
    .withColumn("country", F.element_at(F.col("location"), -1))\
    .withColumn("country", F.regexp_replace(F.col("country"), "[^a-zA-Z0-9 ]", ""))\
    .drop("location")

In [17]:
events = events.withColumnRenamed("#", "event_num")
for _ in events.columns:
    events = events.withColumnRenamed(_, _.lower())

cols = ["event_num", "event", "date", "venue", "city", "state", "country", "attendance"]

events = events.select(*cols)

In [18]:
show(events.filter(F.col("attendance").isNotNull()))

,event_num,event,date,venue,city,state,country,attendance
0,82,Glory: Collision 2,2019-12-21,GelreDome,None,Arnhem,Netherlands,31000
1,49,Glory 41: Holland,2017-05-20,Brabanthallen,None,Den Bosch,Netherlands,5000
2,48,Glory 40: Copenhagen,2017-04-29,Forum Copenhagen,None,Copenhagen,Denmark,2500
3,47,Glory 39: Brussels,2017-03-25,Vorst National,None,Brussels,Belgium,1500
4,43,Glory 36: Oberhausen,2016-12-10,König Pilsener Arena,None,Oberhausen,Germany,13000
